## AudioBooks Tensorflow
### lirbaries to import

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import preprocessing
print('tensorflow version: ',tf.__version__)

tensorflow version:  2.0.0


### Loading the data

#### Extracting the data from .csv file and converting it to .npz file

In [3]:
csv_data = pd.read_csv('../data/Audiobooks-data.csv',header=None)
#csv_data.head()
csv_data = csv_data.iloc[1:]

#without ID and targets columns
input_data = csv_data.iloc[:,1:-1]

#selecting only the targets column
targets = csv_data.iloc[:,-1] 
targets = targets.to_numpy()

### Balancing the Dataset

In [4]:
# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = np.sum(targets)

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(1,targets.shape[0]):
    if int(targets[i]) == 0:
        zero_targets_counter += 1
        if zero_targets_counter > int(num_one_targets):
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = input_data.drop(indices_to_remove,axis=0)
targets_equal_priors = np.delete(targets, indices_to_remove, axis=0)
targets_equal_priors


array(['0', '0', '0', ..., '0', '0', '1'], dtype=object)

### Standardizing the inputs

In [5]:
# standardizing the data using preprocessing functionality from sklearn

scaled_input_data = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffling the data

In [6]:
shuffledindex = np.arange(scaled_input_data.shape[0])
np.random.shuffle(shuffledindex)

#shuffling input data
shuffled_input_data = scaled_input_data[shuffledindex]
shuffled_targets = targets_equal_priors[shuffledindex]

### Splitting the data into Train, Validation and Test dataset

In [7]:
total_sample_count = shuffled_input_data.shape[0]

#dividing the shuffled input data into train, validation and test datasets
# 80% of input data would be training dataset
# 10% of input data would be validation dataset
# remaining 10% would be the test dataset
# Same for the targets dataset
train_count = int(0.8 * total_sample_count)
validation_count = int(0.1 * total_sample_count)
test_count = total_sample_count - train_count - validation_count

#Extracting the data from the input dataset into the respective train, validation and test dataset
train_data = shuffled_input_data[:train_count]
validation_data = shuffled_input_data[train_count:train_count+validation_count]
test_data = shuffled_input_data[train_count+validation_count:]

#Extracting targets data into respective train, validation and test dataset
train_target = shuffled_targets[:train_count]
validation_target = shuffled_targets[train_count:train_count+validation_count]
test_target = shuffled_targets[train_count+validation_count:]

### Saving the data into .npz file to work with tensorflow

In [8]:
np.savez('Audiobooks-data-train', inputs=train_data, targets=train_target)
np.savez('Audiobooks-data-validation', inputs=validation_data, targets=validation_target)
np.savez('Audiobooks-data-test', inputs=test_data, targets=test_target)

## Working with Data

In [9]:
#Extracting the .npz into the temporary variables
#Converting the input data into float and targets to int

# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

#Extracting Train data
npz = np.load('../Audiobook/data/Audiobooks-data-train.npz')

train_input_data = npz['inputs'].astype(np.float)
train_target_data = npz['targets'].astype(np.int)

#Extracting Validation data
npz = np.load('../Audiobook/data/Audiobooks-data-validation.npz')

validation_input_data = npz['inputs'].astype(np.float)
validation_target_data = npz['targets'].astype(np.int)

#Extracting Test data
npz = np.load('../Audiobook/data/Audiobooks-data-test.npz')

test_input_data = npz['inputs'].astype(np.float)
test_target_data = npz['targets'].astype(np.int)


# restore np.load for future normal usage
np.load = np_load_old

### Outlining the model

In [58]:
#Setting the width, depth of the network and the activation function for each hidden and an output layer.
input_size = 10
output_size = 2
hidden_layer_size = 50
    
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is implementing: output = activation(dot(input, weight) + bias)
    # hidden layer 1
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), 
    # hidden layer 2
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    #output layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

### Choosing the optimizer and the loss function

In [59]:
# loss function would be cross entropy as the data is categorical.
# loss method is chosen to be sparse categorical cross entropy as data isnt one hot encoded. 

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Setting the width, depth of the network and the activation function for each hidden and an output layer.

### Training the model

In [61]:
NUM_OF_EPOCHS = 20
BATCH_SIZE = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)
# task here is to check if val_loss is increasing

model.fit(train_input_data,train_target_data,batch_size=BATCH_SIZE,epochs=NUM_OF_EPOCHS,
          callbacks=[early_stopping],validation_data=(validation_input_data, validation_target_data),verbose = 2)  

Train on 11267 samples, validate on 1408 samples
Epoch 1/20
11267/11267 - 0s - loss: 0.2312 - accuracy: 0.9102 - val_loss: 0.2329 - val_accuracy: 0.9084
Epoch 2/20
11267/11267 - 0s - loss: 0.2299 - accuracy: 0.9116 - val_loss: 0.2329 - val_accuracy: 0.9084
Epoch 3/20
11267/11267 - 0s - loss: 0.2269 - accuracy: 0.9105 - val_loss: 0.2353 - val_accuracy: 0.9084


In [63]:
### Testing the model
test_loss, test_accuracy = model.evaluate(test_input_data, test_target_data)

1409/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [64]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))


Test loss: 0.22. Test accuracy: 91.55%
